In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv("../dataset")
x_columns = data.columns.drop('label')
x = data[x_columns]
x = x.values
y = data['label']
le_proto = LabelEncoder()
x[:, 0] = le_proto.fit_transform(x[:, 0])
# le_label = LabelEncoder()
# y = le_label.fit_transform(y)
sc = StandardScaler()
x=sc.fit_transform(x)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [3]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [4]:
# y_pred = knn.predict(x_test)

# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [5]:
datatest = pd.read_csv("../datatest")
x_columns = datatest.columns.drop('label')
x = datatest[x_columns]
x = x.values
y = datatest['label']

x[:, 0] = le_proto.transform(x[:, 0])
x=sc.transform(x)

y_pred = knn.predict(x)
stats = {}
labels = ['normal','tcpsynflood','icmpflood','udpflood','ipsweep','portscan','pingofdeath']
for i in labels:
    stats[i]={}
    for j in labels:
        stats[i][j] = 0
for i in range(len(y)):
    stats[y[i]][y_pred[i]] += 1
stats

{'normal': {'normal': 1219,
  'tcpsynflood': 0,
  'icmpflood': 11,
  'udpflood': 11,
  'ipsweep': 0,
  'portscan': 1,
  'pingofdeath': 0},
 'tcpsynflood': {'normal': 80,
  'tcpsynflood': 961,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 4,
  'pingofdeath': 0},
 'icmpflood': {'normal': 33,
  'tcpsynflood': 0,
  'icmpflood': 760,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'udpflood': {'normal': 2,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 823,
  'ipsweep': 0,
  'portscan': 1,
  'pingofdeath': 0},
 'ipsweep': {'normal': 38,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 612,
  'portscan': 0,
  'pingofdeath': 0},
 'portscan': {'normal': 3,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 3,
  'ipsweep': 0,
  'portscan': 912,
  'pingofdeath': 0},
 'pingofdeath': {'normal': 11,
  'tcpsynflood': 0,
  'icmpflood': 7,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 450}}

In [8]:
tpr={}
fpr={}
precision={}
f1score={}
for i in labels:
    tp = stats[i][i]
    fp = 0
    tn = 0
    fn = 0
    for j in labels:
        if i != j:
            fn += stats[i][j]
            fp += stats[j][i]
            for k in stats[j].keys():
                if k != i:
                    tn += stats[j][k]
    tpr[i] = tp / (tp + fn) * 100
    fpr[i] = fp / (fp + tn) * 100
    precision[i] = tp / (tp + fp) * 100
    f1score[i] = 2/(1/tpr[i] + 1/precision[i])
print('TPR: \n ' + str(tpr))
print('FRP:\n ' + str(fpr))
# print('Precision:\n ' + str(precision))
print('F1-score: \n' + str(f1score))

print("Accuracy:",metrics.accuracy_score(y, y_pred))

TPR: 
 {'normal': 98.14814814814815, 'tcpsynflood': 91.96172248803828, 'icmpflood': 95.83858764186634, 'udpflood': 99.63680387409201, 'ipsweep': 94.15384615384616, 'portscan': 99.34640522875817, 'pingofdeath': 96.15384615384616}
FRP:
 {'normal': 3.553191489361702, 'tcpsynflood': 0.0, 'icmpflood': 0.349582443192853, 'udpflood': 0.2736512900703675, 'ipsweep': 0.0, 'portscan': 0.11942675159235669, 'pingofdeath': 0.0}
F1-score: 
{'normal': 92.77016742770167, 'tcpsynflood': 95.81256231306081, 'icmpflood': 96.75366008911521, 'udpflood': 98.97775105231509, 'ipsweep': 96.98890649762282, 'portscan': 99.34640522875817, 'pingofdeath': 98.0392156862745}
Accuracy: 0.9654998317064961
